<a href="https://colab.research.google.com/github/GustaveRw/MINECOFIN-Feedback-Analysis/blob/master/MINECOFIN_Feedback_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snscrape
!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 KB 2.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-njw2rgkg
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-njw2rgkg
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit 03ef3debaf9c32d20cf4ff557a24276c34f12005
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for UNKNOWN: filename=UNKNOWN-0.5.0.20230114.dev35+g03ef3de-py3-none-any.whl size=13561 sha256=3a1ec5d13ca9dab8d385502a9c8433730e69effc6f56556beb0235f6db9938a3
  Stored in directory: 

In [2]:
import pandas as pd 

import numpy as np 

import snscrape.modules.twitter as sntwitter 

import datetime 

from tqdm.notebook import tqdm_notebook 

import seaborn as sns 

import matplotlib.pyplot as plt 

sns.set_theme(style="whitegrid") 

In [6]:
text = input('Enter query text to be matched (or leave it blank by pressing enter)') 

username = input('Enter specific username(s) from a twitter account without @ (or leave it blank by pressing enter): ') 

since = input('Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): ') 

until = input('Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): ') 

count = (input('Enter max number of tweets or enter -1 to retrieve all possible tweets: ')) 

retweet = input('Exclude Retweets? (y/n): ') 

replies = input('Exclude Replies? (y/n): ')

Enter query text to be matched (or leave it blank by pressing enter)finance
Enter specific username(s) from a twitter account without @ (or leave it blank by pressing enter): RwandaFinance
Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): 
Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): 
Enter max number of tweets or enter -1 to retrieve all possible tweets: -1
Exclude Retweets? (y/n): y
Exclude Replies? (y/n): y


In [4]:
import datetime

def search(text, username='', since='', until='', retweet='n', replies='n'):
    global filename
    
    # Convert tuple arguments to strings
    if isinstance(text, tuple):
        text = ' '.join(text)
    if isinstance(username, tuple):
        username = ' '.join(username)
    if isinstance(since, tuple):
        since = ' '.join(since)
    if isinstance(until, tuple):
        until = ' '.join(until)
    
    q = text

    if username:
        q += f' from:{username}'

    if not since:
        since = (datetime.date.today() - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
    q += f' since:{since}'

    if not until:
        until = datetime.date.today().strftime('%Y-%m-%d')
    q += f' until:{until}'

    if retweet == 'n':
        q += ' exclude:retweets'

    if replies == 'n':
        q += ' exclude:replies'

    # Generate filename based on search parameters
    if username and text:
        filename = f'{since}_{until}_{username}_{text}.csv'
    elif username:
        filename = f'{since}_{until}_{username}.csv'
    else:
        filename = f'{since}_{until}_{text}.csv'

    print(filename)
    return q


In [7]:
from tqdm.notebook import tqdm_notebook

q = search(text, username, since, until, retweet, replies)

# Creating list to append tweet data
tweets_list1 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
if count == -1:
    for i, tweet in enumerate(tqdm_notebook(sntwitter.TwitterSearchScraper(q).get_items())):
        tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username, tweet.lang, tweet.hashtags, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount, tweet.media])
else:
    with tqdm_notebook(total=count) as pbar:
        for i, tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()):
            if i >= {count}:
                break
            tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username, tweet.lang, tweet.hashtags, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount, tweet.media])
            pbar.update(1)

# Creating a DataFrame from the tweets list above
tweets_df1 = pd.DataFrame(tweets_list1, columns=['DateTime', 'TweetId', 'Text', 'Username', 'Language', 'Hashtags', 'ReplyCount', 'RetweetCount', 'LikeCount', 'QuoteCount', 'Media'])

# Dropping duplicates based on tweet ID
tweets_df1.drop_duplicates(subset='TweetId', keep='first', inplace=True)

# Resetting index of the DataFrame
tweets_df1.reset_index(drop=True, inplace=True)

# Converting 'DateTime' column to datetime format
tweets_df1['DateTime'] = pd.to_datetime(tweets_df1['DateTime'])

# Setting 'TweetId' column as the index
tweets_df1.set_index('TweetId', inplace=True)

# Sorting the DataFrame by 'DateTime' column
tweets_df1.sort_values(by='DateTime', inplace=True)

# Printing the first 5 rows of the DataFrame
print(tweets_df1.head())


2023-02-23_2023-03-02_RwandaFinance_finance.csv


0it [00:00, ?it/s]

                                     DateTime  \
TweetId                                         
1628701244805066753 2023-02-23 10:20:11+00:00   
1629054408880267265 2023-02-24 09:43:32+00:00   
1629054413024141312 2023-02-24 09:43:33+00:00   
1630915452409896960 2023-03-01 12:58:39+00:00   

                                                                  Text  \
TweetId                                                                  
1628701244805066753  A public-private policy dialogue on the status...   
1629054408880267265  Minister @undagijimana today held bilateral ta...   
1629054413024141312  They also signed a memorandum of understanding...   
1630915452409896960  Watch Minister of State in Charge of National ...   

                          Username Language  Hashtags  ReplyCount  \
TweetId                                                             
1628701244805066753  RwandaFinance       en      None           0   
1629054408880267265  RwandaFinance       en      None